## Dependencies
* standard libraries: random, math, itertools
* numpy scipy
* keras / tensorflow

In [1]:
import random
import math
from itertools import permutations
import numpy as np
from scipy.spatial.transform import Rotation as R

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
print(K.backend())

Using TensorFlow backend.


tensorflow


## Model

In [3]:
all_rotation_orders = [''.join(axes) for axes in permutations({'x','y','z'})]
print(all_rotation_orders)

['yzx', 'yxz', 'zyx', 'zxy', 'xyz', 'xzy']


In [4]:
def random_vector(n, mag):
    return 2.0 * mag * (0.5-np.random.random(size=n))
print(random_vector(10,5.0))

[-3.24972153  4.30385362  0.48243849  4.52010599 -3.12602618  0.74478975
 -1.27703979 -4.02305146 -1.40080634  0.2750498 ]


define a function to generate a single sample from the model.

sample is a tuple of:
* np.array with 9 matrix elements + 3 rotation angles
* string corresponding to chosen rotation order (element of all_rotation_orders)

In [5]:
def generate_sample_from_model():
    rotation_order = random.choice(all_rotation_orders)  # choose a rotation order
    angles = random_vector(3, 10.0)   # generate three random angles (in degrees)
    if (min(abs(angles)) < 0.1):     # reject any angles that are too small
        return generate_sample_from_model()
    matrix = R.from_euler(rotation_order, angles, degrees = True)  # generate matrix
    noise_angles = angles + random_vector(3, 0.1)    
    x = np.concatenate((matrix.as_dcm().flatten(), 
                        np.array(noise_angles)))
                        # np.array(np.cos(noise_angles)), 
                        # np.array(np.sin(noise_angles))))                         
    return (x, rotation_order)
print(generate_sample_from_model())

(array([ 0.99935585,  0.03466508,  0.00928567, -0.03379935,  0.99613019,
       -0.08113112, -0.01206216,  0.08076501,  0.99666018,  4.57420847,
       -1.85894155,  0.76948431]), 'xzy')


define a function to generate multiple samples, suitable for training or testing.

for number N, sample is a tuple:
 * np.array Nx12 of x vectors (standardized zero mean)
 * np.array Nx6 one-hot vectors

In [6]:
def generate_samples(n, mean=None, std=None):
    [xs, rotation_orders] = list(zip(*[generate_sample_from_model() for _ in range(n)]))
    xs = np.array(xs)
    if mean is None:
        mean = xs.mean(0)
    if std is None:
        std = xs.std(0)
    xs -= mean    # standardize along sample dimension
    xs /= std
    ys = [all_rotation_orders.index(rotation_order) for rotation_order in rotation_orders]
    return xs, mean, std, keras.utils.to_categorical(ys)
print(generate_samples(3))

(array([[ 0.43562384,  0.35618446, -0.64646539, -0.35623185,  0.33139478,
        -0.66548978,  0.5273733 ,  0.72190244,  1.15743544,  0.07408695,
         0.78642897, -1.11418767],
       [-1.38300457,  1.00717123, -0.76606151, -1.00713685, -1.35634139,
         1.41341251,  0.87271471, -1.41410964, -1.28246299, -1.26010657,
        -1.4111276 , -0.19718687],
       [ 0.94738072, -1.36335569,  1.4125269 ,  1.3633687 ,  1.02494661,
        -0.74792274, -1.40008801,  0.6922072 ,  0.12502754,  1.18601962,
         0.62469863,  1.31137454]]), array([ 0.99219216,  0.09408371,  0.02216775, -0.09457565,  0.98646436,
        0.11756445, -0.00956203, -0.11594338,  0.9904732 , -6.72500726,
       -4.09676622, -0.24791605]), array([2.73666248e-03, 5.59795791e-02, 5.54109069e-02, 4.98078363e-02,
       9.06985384e-03, 3.94845131e-02, 6.34856673e-02, 3.71143535e-02,
       4.54257475e-03, 1.34662230e+00, 4.15555771e+00, 4.44987053e+00]), array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.,

generate training and testing data from the model

In [7]:
x_train, mean, std, y_train = generate_samples(10000)
x_test, _, _, y_test = generate_samples(1000, mean, std)
print(x_train.shape, y_train.shape)

(10000, 12) (10000, 6)


now create an MLP as the model

In [8]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = x_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 4,166
Trainable params: 4,166
Non-trainable params: 0
_________________________________________________________________


set up the optimizer and compile the model

In [9]:
adam = Adam(lr=0.001, decay=1e-8)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

now fit the data using the training samples

In [10]:
model.fit(x_train, y_train, epochs=200, batch_size=256)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
10000/10000 [==============================] - 0s 24us/step - loss: 1.6787 - acc: 0.3182
Epoch 2/200
10000/10000 [==============================] - 0s 5us/step - loss: 1.3758 - acc: 0.5111
Epoch 3/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.9808 - acc: 0.6928
Epoch 4/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.6703 - acc: 0.7938
Epoch 5/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.4959 - acc: 0.8477
Epoch 6/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.3998 - acc: 0.8812
Epoch 7/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.3352 - acc: 0.9014
Epoch 8/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.2953 - acc: 0.9153
Epoch 9/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.2611 - acc: 0.9282
Epoch 10/200
10000/10000 [=========================

10000/10000 [==============================] - 0s 5us/step - loss: 0.0405 - acc: 0.9884
Epoch 80/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0358 - acc: 0.9923
Epoch 81/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0368 - acc: 0.9908
Epoch 82/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0349 - acc: 0.9917
Epoch 83/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0359 - acc: 0.9919
Epoch 84/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0346 - acc: 0.9915
Epoch 85/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0351 - acc: 0.9917
Epoch 86/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0532 - acc: 0.9829
Epoch 87/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0387 - acc: 0.9886
Epoch 88/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0410 - acc: 0.9887
Epo

10000/10000 [==============================] - 0s 6us/step - loss: 0.0197 - acc: 0.9948
Epoch 160/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0191 - acc: 0.9949
Epoch 161/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0228 - acc: 0.9930
Epoch 162/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0209 - acc: 0.9939
Epoch 163/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0298 - acc: 0.9911
Epoch 164/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0246 - acc: 0.9914
Epoch 165/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0298 - acc: 0.9886
Epoch 166/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0223 - acc: 0.9931
Epoch 167/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0247 - acc: 0.9910
Epoch 168/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0358 - acc: 0

In [11]:
score = model.evaluate(x_test, y_test, batch_size=256)
print(score)

1000/1000 [==============================] - 0s 33us/step
[0.04175947864353657, 0.9839999966621399]


In [12]:
import json
import csv
with open('model_config.json', 'w') as modelfile:  
    json.dump(model.get_config(), modelfile)

In [13]:
def rows_of(array):
    if len(array.shape) > 1:
        for n in range(array.shape[0]):
            yield (n, array[n])
    else:
        yield (0, array)
        
with open('weights.csv', 'w', newline='') as csvfile:
    weightswriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for layer in model.layers:
        for weight_variable in layer.weights:
            weight_values = K.batch_get_value(weight_variable)
            for (row_num, row_array) in rows_of(weight_values):
                if len(weight_values.shape) > 1:
                    row_count = weight_values.shape[0]
                    col_count = weight_values.shape[1]
                else:
                    row_count = 1
                    col_count = weight_values.shape[0]                    
                csv_row = [layer.name, weight_variable.name,
                           row_count, col_count, row_num] + list(row_array)
                weightswriter.writerow(csv_row)

In [14]:
!jupyter nbconvert --to python train_rotation_order.ipynb --template=python_script.tpl

[NbConvertApp] Converting notebook train_rotation_order.ipynb to python
[NbConvertApp] Writing 4554 bytes to train_rotation_order.py
